In [1]:
#import for performance check
import timeit
from time import time
start_pynb = timeit.default_timer()

In [2]:
#import important packages
import json
import numpy as np
import pandas as pd

In [3]:
#read the source data (json format)
df_source = pd.read_json('DataFile.txt')

In [4]:
#save source in csv format
df_source.to_csv("DataSource.csv", index=False)

In [5]:
#save source in json format
df_source.to_json("DataSource.json", orient = "records")

In [6]:
#display source dataframe
df_source

,Gender,HeightCm,WeightKg
0,Male,171,96
1,Male,161,85
2,Male,180,77
3,Female,166,62
4,Female,150,70
5,Female,167,82


In [7]:
#read reference data
df_reference = pd.read_csv("LookupTable.csv")

In [8]:
#Add CatCode as index
df_reference['CatCode'] = df_reference.index
df_reference.insert(0, 'CatCode', df_reference.pop('CatCode'))
df_reference = df_reference.set_index('CatCode')

In [9]:
#display reference dataframe
df_reference

,Category,RangeStart,RangeEnd,Risk
CatCode,,,,
0,Underweight,0.0,18.4,Malnutrition risk
1,Normal weight,18.5,24.9,Low risk
2,Overweight,25.0,29.9,Enhanced risk
3,Moderately obese,30.0,34.9,Medium risk
4,Severely obese,35.0,39.9,High risk
5,Very severely obese,40.0,99.0,Very high risk


In [10]:
type(df_reference)

pandas.core.frame.DataFrame

In [11]:
#make a copy of source dataframe
df_additional = df_source.copy()

In [12]:
#display dataframe with additional info
df_additional

,Gender,HeightCm,WeightKg
0,Male,171,96
1,Male,161,85
2,Male,180,77
3,Female,166,62
4,Female,150,70
5,Female,167,82


In [13]:
#Convert Height to m**2
df_additional['Heightm2'] = round((df_additional['HeightCm'] / 100) ** 2, 2)

In [14]:
#DerivedValue  = WeightKg / Heightm2
df_additional['DerivedValue'] =  round(df_additional ['WeightKg'] / df_additional['Heightm2'], 2)

In [15]:
#display dataframe with additional info
df_additional

,Gender,HeightCm,WeightKg,Heightm2,DerivedValue
0,Male,171,96,2.92,32.88
1,Male,161,85,2.59,32.82
2,Male,180,77,3.24,23.77
3,Female,166,62,2.76,22.46
4,Female,150,70,2.25,31.11
5,Female,167,82,2.79,29.39


In [16]:
#set CatCode dataframe with reference data values for which condition is satisfied 
df_additional.loc[(df_additional.DerivedValue < 18.5), 'CatCode'] = 0
df_additional.loc[(df_additional.DerivedValue >= 18.5) & (df_additional.DerivedValue < 25), 'CatCode'] = 1
df_additional.loc[(df_additional.DerivedValue >= 25) & (df_additional.DerivedValue < 30), 'CatCode'] = 2
df_additional.loc[(df_additional.DerivedValue >= 30) & (df_additional.DerivedValue < 35), 'CatCode'] = 3
df_additional.loc[(df_additional.DerivedValue >= 35) & (df_additional.DerivedValue < 40), 'CatCode'] = 4
df_additional.loc[(df_additional.DerivedValue >= 40), 'CatCode'] = 5
df_additional['CatCode'] = df_additional['CatCode'].astype(int)
df_additional

,Gender,HeightCm,WeightKg,Heightm2,DerivedValue,CatCode
0,Male,171,96,2.92,32.88,3
1,Male,161,85,2.59,32.82,3
2,Male,180,77,3.24,23.77,1
3,Female,166,62,2.76,22.46,1
4,Female,150,70,2.25,31.11,3
5,Female,167,82,2.79,29.39,2


In [17]:
#merge the dataframe with additional info and reference dataframe
df_additional = pd.merge(df_additional, df_reference, how = 'left', on = 'CatCode')

In [18]:
#display dataframe with additional info
df_additional

,Gender,HeightCm,WeightKg,Heightm2,DerivedValue,CatCode,Category,RangeStart,RangeEnd,Risk
0,Male,171,96,2.92,32.88,3,Moderately obese,30.0,34.9,Medium risk
1,Male,161,85,2.59,32.82,3,Moderately obese,30.0,34.9,Medium risk
2,Male,180,77,3.24,23.77,1,Normal weight,18.5,24.9,Low risk
3,Female,166,62,2.76,22.46,1,Normal weight,18.5,24.9,Low risk
4,Female,150,70,2.25,31.11,3,Moderately obese,30.0,34.9,Medium risk
5,Female,167,82,2.79,29.39,2,Overweight,25.0,29.9,Enhanced risk


In [19]:
#save dataframe with additional info in csv format
df_additional.to_csv("DataAdditional.csv", index=False)

In [20]:
#save dataframe with additional info in json format
df_additional.to_json("DataAdditional.json", orient = "records")

In [21]:
#stop timer for performance check
stop_pynb = timeit.default_timer()
print('Time taken: {} seconds'.format(round(stop_pynb - start_pynb,2))) 

Time taken: 1.32 seconds
